In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
from KVClusterV2 import KVClusterV2

In [2]:
# Train
train_directory = '/home/jiaxi/cs525/Experiment/prediction_model/clustering/train_set'
num_clusters = 10

In [3]:
colors = [
    '#e6194B',
    '#f58231',
    '#9A6324',
    '#911eb4',
    '#3cb44b',
    '#f032e6',
    '#4363d8',
]

In [4]:
def load_json_file(file_path):
	try:
		with open(file_path, 'r') as file:
			data = json.load(file)
		return data
	
	except Exception as e:
		print(f"An error occurred while loading the JSON file: {e}")
		return None

In [5]:
def extract_fpss(metric_list):
	return list(metric_list[list(metric_list.keys())[0]][0]['metric'].keys())

In [6]:
def to_accuracy_vector(accuracy_result_seq, fpss, type='F1'):
    accuracy_vector = []
    for fps in fpss:
        accuracy_vector.append(accuracy_result_seq[fps][type])
    
    return accuracy_vector

## Train

In [7]:
train_filenames = sorted(os.listdir(train_directory))
train_video_names = sorted(list(set([f.split('_')[0] for f in train_filenames])))

In [8]:
fpss = extract_fpss(load_json_file(os.path.join(train_directory, train_video_names[0] + "_Accuracy_Result.json")))

cluster = KVClusterV2(num_clusters)

In [9]:
for v in train_video_names:
	accuracy_result = load_json_file(os.path.join(train_directory, v + "_Accuracy_Result.json"))
	movement_result = load_json_file(os.path.join(train_directory, v + "_Movement_Result.json"))

	for class_idx in list(accuracy_result.keys()):
		for i in range(len(accuracy_result[class_idx])):
			accuracy_vector = to_accuracy_vector(accuracy_result[class_idx][i]['metric'], fpss)

			for fps in fpss:
				movement_vector = movement_result[class_idx][i]['movement'][fps]

				cluster.add(movement_vector, accuracy_vector)

In [10]:
cluster.cluster_average()

In [11]:
cluster.save_model('/home/jiaxi/cs525/Experiment/prediction_model/clustering/train_result.json')